# Advent of Code 2024 Day 6 

### Setup

First we need to instantiate some control variables and read in our example and testing data.

In [ ]:
from aocd import get_data, submit

day = 6
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

Both the test and sample data will be stored as a string. The string is unique to the given problem so we will need to implement parse_data in order to store it in a data structure that is useful!

In [ ]:
def parse_data(raw_data):
    return raw_data.split('\n') # you should parse data here

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

### Part One!

In [ ]:
use_sample_data = True
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

In [ ]:
def get_col(grid:list[list[str]], i:int):
    return [ row[i] for row in grid ]

def get_cols(grid: list[list[str]]):
    cols = []
    for i in range(len(grid)):
        cols.append(get_col(grid, i))

    return cols

In [ ]:
contains_guard = lambda x: ('<' in x) or ('^' in x) or ('>' in x) or ('V' in x)

starting_row = [ i for i, row in enumerate(data) if contains_guard(row) ]
starting_col = [ j for j, col in enumerate(get_cols(data)) if contains_guard(col)]

start = (starting_row[0], starting_col[0])
start_direction = data[start[0]][start[1]]

start, start_direction

In [ ]:
def escape_maze(maze:list[list[str]], start:tuple[int, int], start_direction, max_iter=1000):
    ans = {'path' : [], 'visited': set([]), 'visited_count': 0, 'iters': 0, 'turns': {}, 'obstacles': set([])}
    escaped = False
    i = 0
    direction = start_direction
    row, col = start
    max_row, max_col = len(maze) - 1, len(maze[-1]) - 1

    while not escaped and i < max_iter:
        i += 1
        ans['iters'] += 1
        ans['path'].append((row, col, i, direction))

        if (row, col) not in ans['visited']:
            ans['visited'].add((row, col))
            ans['visited_count'] += 1

        if direction == '<':
            next_col = col - 1

            # Check for escape
            if (next_col < 0):
                escaped = True
                break

            # Check for obstacle 
            if maze[row][next_col] == '#':
                # turn right 
                direction = '^'
                ans['obstacles'].add((row, next_col))
                ans['turns'][(row, next_col)] = i if (row, next_col) not in ans['turns'] else ans['turns'][(row, next_col)]

            
            # otherwise move up 
            else:
                col = next_col

        elif direction == '>':
            next_col = col + 1

            if next_col > max_col:
                escaped = True
                break

            if maze[row][next_col] == '#':
                direction = 'V'
                ans['obstacles'].add((row, next_col))
                ans['turns'][(row, next_col)] = i if (row, next_col) not in ans['turns'] else ans['turns'][(row, next_col)]

            else:
                col = next_col
                
        elif direction == 'V':
            next_row = row + 1

            if (next_row > max_row):
                escaped = True
                break

            if maze[next_row][col] == '#':
                # turn right 
                direction = '<'
                ans['obstacles'].add((next_row, col))
                ans['turns'][(next_row, col)] = i if (next_row, col) not in ans['turns'] else ans['turns'][(next_row, col)] 
            
            else:
                row = next_row
                
        elif direction == '^':
            next_row = row - 1

            if next_row < 0:
                escaped = True
                break

            if maze[next_row][col] == '#':
                direction = '>'
                ans['obstacles'].add((next_row, col))
                ans['turns'][(next_row, col)] = i if (next_row, col) not in ans['turns'] else ans['turns'][(next_row, col)] 

            else:
                row = next_row

    return ans

In [ ]:
part_a_answer = escape_maze(data, start, start_direction, max_iter=100_000)

part_a_answer['visited_count']

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer['visited_count'], part='a', day=day, year=year, reopen=True)

### Part Two!

In [ ]:
use_sample_data = False
part='b'

In [ ]:
data = sample_data if use_sample_data else test_data

In [ ]:
starting_row = [ i for i, row in enumerate(data) if contains_guard(row) ]
starting_col = [ j for j, col in enumerate(get_cols(data)) if contains_guard(col)]

start = (starting_row[0], starting_col[0])
start_direction = data[start[0]][start[1]]

start, start_direction

In [ ]:
def identify_weakspots(maze:list[list[str]], path:list[tuple[int, int, int, str]], obstacles: set[tuple[int, int]], turns: dict[tuple[int, int]: int], **kwargs):
    weakspots = set([])
    for row_idx, col_idx, i, direction in path:
        search_space = []

        # get search space
        if direction == '^':
            search_space = [ (row_idx, i) for i in range(len(maze[row_idx])) if i > col_idx ]
        
        elif direction == '>':
            search_space = [ (i, col_idx) for i in range(len(get_col(maze, col_idx))) if i > row_idx ]

        elif direction == 'V':
            search_space = [ (row_idx, i) for i in range(len(maze[row_idx])) if i < col_idx ]
        
        elif direction == '<':
            search_space = [ (i, col_idx) for i in range(len(get_col(maze, col_idx))) if i < row_idx ]
        
        # determine if there is an obstalce in the search space that we have already hit
        if any([ i < turns[x] for x in search_space if x in obstacles]):
            weakspots.add((row_idx, col_idx))
        
    return {'weakspots': weakspots, "count": len(weakspots)}

In [ ]:
part_b_answer = 0

max_iter = 100_000
l = 100
for i in range(len(data)):
    print(f"Still Running! {len(data) - i} more rows to go!")
    for j in range(len(data[0])):
        if data[i][j] == '#' or contains_guard(data[i][j]):
            continue
        
        candidate = [ list(row[:]) for row in data ]
        candidate[i][j] = '#'

        result = escape_maze(candidate, start=start, start_direction=start_direction, max_iter=max_iter)

        if result['iters'] >= max_iter - l:
            part_b_answer += 1

part_b_answer


In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)